In [ ]:
"""Input Format. Input contains one string 𝑆 which consists of big and small latin letters, digits, punctuation
marks and brackets from the set []{}().

Constraints. The length of 𝑆 is at least 1 and at most 10**5

Output Format. If the code in 𝑆 uses brackets correctly, output “Success" (without the quotes). Otherwise,
output the 1-based index of the first unmatched closing bracket, and if there are no unmatched closing
brackets, output the 1-based index of the first unmatched opening bracket.

Sample 1.
Input:
[]
Output:
Success

Sample 6.
Input:
{
Output:
1

Sample 9.
Input:
foo(bar[i);
Output:
10
Explanation:
) doesn’t match [, so ) is the first unmatched closing bracket, so we output its position, which is 10."""

In [6]:
from collections import namedtuple
Bracket = namedtuple("Bracket",["char","position"])
def find_mismatch(text):
    opening_brackets_stack = []
    for i,next in enumerate(text):
        if next in "([{":
            bracket = Bracket(next,i)
            opening_brackets_stack.append(bracket)
        if next in ")]}":
            if not opening_brackets_stack:
                return i + 1
            top = opening_brackets_stack.pop()
            if (top.char == '(' and next != ')') or (top.char == '[' and next != ']') or (top.char == '{' and next != '}'):
                return i + 1 
    if opening_brackets_stack:
        return opening_brackets_stack[0].position + 1 
    
    return "Success"

def main():
    text = input()
    mismatch = find_mismatch(text)
    print(mismatch)

if __name__ == "__main__":
    main()


Success


In [ ]:
"""Task. You are given a description of a rooted tree. Your task is to compute and output its height. Recall
that the height of a (rooted) tree is the maximum depth of a node, or the maximum distance from a
leaf to the root. You are given an arbitrary tree, not necessarily a binary tree.

Input Format. The first line contains the number of nodes 𝑛. The second line contains 𝑛 integer numbers
from −1 to 𝑛 − 1 — parents of nodes. If the 𝑖-th one of them (0 ≤ 𝑖 ≤ 𝑛 − 1) is −1, node 𝑖 is the root,
otherwise it’s 0-based index of the parent of 𝑖-th node. It is guaranteed that there is exactly one root.
It is guaranteed that the input represents a tree.

Constraints. 1 ≤ 𝑛 ≤ 10**5

Output Format. Output the height of the tree.

Sample 1.
Input:
5
4 -1 4 1 1
Output:
3

Sample 2.
Input:
5
-1 0 4 0 3
Output:
4"""


In [ ]:
import sys
import threading

def compute_height(n, parents):
    # Replace this code with a faster implementation
    max_height = 0
    for vertex in range(n):
        height = 0
        current = vertex
        while current != -1:
            height += 1
            current = parents[current]
        max_height = max(max_height, height)
    return max_height


def main():
    n = int(input())
    parents = list(map(int, input().split()))
    print(compute_height(n, parents))


# In Python, the default limit on recursion depth is rather low,
# so raise it here for this problem. Note that to take advantage
# of bigger stack, we have to launch the computation in a new thread.
sys.setrecursionlimit(10**7)  # max depth of recursion
threading.stack_size(2**27)   # new thread will get stack of such size
threading.Thread(target=main).start()

In [ ]:
"""Task. You are given a series of incoming network packets, and your task is to simulate their processing.
Packets arrive in some order. For each packet number 𝑖, you know the time when it arrived 𝐴𝑖 and the
time it takes the processor to process it 𝑃𝑖 (both in milliseconds). There is only one processor, and it
processes the incoming packets in the order of their arrival. If the processor started to process some
packet, it doesn’t interrupt or stop until it finishes the processing of this packet, and the processing of
packet 𝑖 takes exactly 𝑃𝑖 milliseconds.
The computer processing the packets has a network buffer of fixed size 𝑆. When packets arrive, they are stored in the buffer before being processed. However, if the buffer is full when a packet
arrives (there are 𝑆 packets which have arrived before this packet, and the computer hasn’t finished
processing any of them), it is dropped and won’t be processed at all. If several packets arrive at the
same time, they are first all stored in the buffer (some of them may be dropped because of that —
those which are described later in the input). The computer processes the packets in the order of
their arrival, and it starts processing the next available packet from the buffer as soon as it finishes
processing the previous one. If at some point the computer is not busy, and there are no packets in
the buffer, the computer just waits for the next packet to arrive. Note that a packet leaves the buffer
and frees the space in the buffer as soon as the computer finishes processing it.

Input Format. The first line of the input contains the size 𝑆 of the buffer and the number 𝑛 of incoming
network packets. Each of the next 𝑛 lines contains two numbers. 𝑖-th line contains the time of arrival
𝐴𝑖 and the processing time 𝑃𝑖 (both in milliseconds) of the 𝑖-th packet. It is guaranteed that the
sequence of arrival times is non-decreasing (however, it can contain the exact same times of arrival in
milliseconds — in this case the packet which is earlier in the input is considered to have arrived earlier).

Constraints. All the numbers in the input are integers. 1 ≤ 𝑆 ≤ 10**5; 0 ≤ 𝑛 ≤ 10**5; 0 ≤ 𝐴𝑖 ≤ 10**6;0 ≤ 𝑃𝑖 ≤ 10**3
; 𝐴𝑖 ≤ 𝐴𝑖+1 for 1 ≤ 𝑖 ≤ 𝑛 − 1.

Output Format. For each packet output either the moment of time (in milliseconds) when the processor
began processing it or −1 if the packet was dropped (output the answers for the packets in the same
order as the packets are given in the input).

Sample 1.
Input:
1 0
Output:

Sample 2.
Input:
1 1
0 0
Output:
0
Explanation:
The only packet arrived at time 0, and computer started processing it immediately.


Sample 3.
Input:
1 2
0 1
0 1
Output:
0
-1
Explanation:
The first packet arrived at time 0, the second packet also arrived at time 0, but was dropped, because
the network buffer has size 1 and it was full with the first packet already. The first packet started
processing at time 0, and the second packet was not processed at all.
"""

In [18]:
from collections import namedtuple
import collections


Request = namedtuple("Request", ["arrived_at", "time_to_process"])
Response = namedtuple("Response", ["was_dropped", "started_at"])


class Buffer:
    def __init__(self, size):
        self.size = size
        self.finish_time = collections.deque(maxlen=size)

    def process(self, request):
        while len(self.finish_time) > 0:
            if self.finish_time[0] <= request.arrived_at:
                self.finish_time.popleft()
            else:
                break
        packets = len(self.finish_time)
        if packets == self.size:
            return Response(True, -1)
        if packets == 0:
            started_at = request.arrived_at
        else:
            started_at = self.finish_time[packets - 1]
        finished_at = started_at + request.time_to_process
        self.finish_time.append(finished_at)
        return Response(False, started_at)


def process_requests(requests, buffer):
    responses = []
    for request in requests:
        responses.append(buffer.process(request))
    return responses


def main():
    buffer_size, n_requests = map(int, input().split())
    requests = []
    for _ in range(n_requests):
        arrived_at, time_to_process = map(int, input().split())
        requests.append(Request(arrived_at, time_to_process))

    buffer = Buffer(buffer_size)
    responses = process_requests(requests, buffer)

    for response in responses:
        print(response.started_at if not response.was_dropped else -1)


if __name__ == "__main__":
    main()

0
-1


In [ ]:
"""Task. Implement a stack supporting the operations Push(), Pop(), and Max().

Input Format. The first line of the input contains the number 𝑞 of queries. Each of the following 𝑞 lines
specifies a query of one of the following formats: push v, pop, or max.

Constraints. 1 ≤ 𝑞 ≤ 400 000, 0 ≤ 𝑣 ≤ 105
.
Output Format. For each max query, output (on a separate line) the maximum value of the stack.

Sample 1.
Input:
5
push 2
push 1
max
pop
max
Output:
2
2
Explanation:
After the first two push queries, the stack contains elements 1 and 2. After the pop query, the element
1 is removed.

Sample 2.
Input:
5
push 1
push 2
max
pop
max
Output:
2
1

Sample 3.
Input:
10
push 2
push 3
push 9
push 7
push 2
max
max
max
pop
max
Output:
9
9
9
9
"""

In [ ]:
import sys


class StackWithMax():
    def __init__(self):
        self.__stack = []
        self.max_value = []

    def Push(self, a):
        if not self.max_value:
            self.max_value.append(a)
        elif a >= self.max_value[-1]:
            self.max_value.append(a)
        self.__stack.append(a)

    def Pop(self):
        assert(len(self.__stack))
        if self.__stack[-1] == self.max_value[-1]:
            self.max_value.pop()
        self.__stack.pop()

    def Max(self):
        return self.max_value[-1]


if __name__ == '__main__':
    stack = StackWithMax()

    num_queries = int(sys.stdin.readline())
    for _ in range(num_queries):
        query = sys.stdin.readline().split()

        if query[0] == "push":
            stack.Push(int(query[1]))
        elif query[0] == "pop":
            stack.Pop()
        elif query[0] == "max":
            print(stack.Max())
        else:
            assert(0)

In [ ]:
"""Problem Introduction
Given a sequence 𝑎1, . . . , 𝑎𝑛 of integers and an integer 𝑚 ≤ 𝑛, find the maximum among {𝑎𝑖
, . . . , 𝑎𝑖+𝑚−1} for every 1 ≤ 𝑖 ≤ 𝑛 − 𝑚 + 1. A naive 𝑂(𝑛𝑚) algorithm for solving this problem scans each window separately.
Your goal is to design an 𝑂(𝑛) algorithm.

Problem Description

Input Format. The first line contains an integer 𝑛, the second line contains 𝑛 integers 𝑎1, . . . , 𝑎𝑛 separated
by spaces, the third line contains an integer 𝑚.

Constraints. 1 ≤ 𝑛 ≤ 105, 1 ≤ 𝑚 ≤ 𝑛, 0 ≤ 𝑎𝑖 ≤ 10**5 for all 1 ≤ 𝑖 ≤ 𝑛.

Output Format. Output max{𝑎𝑖, . . . , 𝑎𝑖+𝑚−1} for every 1 ≤ 𝑖 ≤ 𝑛 − 𝑚 + 1.

Sample 1.
Input:
8
2 7 3 1 5 2 6 2
4
Output:
7 7 5 6 6"""

In [6]:
from collections import deque


def maxSlidingWindow(sequence, m):
    dq = deque()
    max_nums = []
    length = len(sequence)
    for i in range(length):
        while dq and sequence[i] >= sequence[dq[-1]]:
            dq.pop()
        dq.append(i)
        if i >= m and dq and dq[0] == i - m:
            dq.popleft()
        if i >= m - 1:
            max_nums.append(sequence[dq[0]])
    return max_nums


if __name__ == '__main__':
    n = int(input())
    input_sequence = [int(i) for i in input().split()]
    assert len(input_sequence) == n
    window_size = int(input())
    print(*maxSlidingWindow(input_sequence, window_size))

3
